In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [15]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse import rand

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [2]:
def mat_mul_coo(A, B):
   
    assert A.shape[1] == B.shape[0], "Matrices cannot be multiplied"
    
    data = []
    row = []
    col = []
    
    
    dict_B = {} #dic to store the non-zero elements of each row in B
    for j, val in zip(B.row, B.data):
        if j not in dict_B:
            dict_B[j] = []
        dict_B[j].append(val)
    
    for i, a_val in zip(A.row, A.data):
        if a_val == 0:
            continue
        
        #get non zero elements and column indices of this row in B
        if i in dict_B:
            b_vals = dict_B[i]
            b_cols = B.col[B.row == i]
        else:
            b_vals = []
            b_cols = []
        
        #dot product
        for b_val, b_col in zip(b_vals, b_cols):
            c_val = a_val * b_val
            c_row = i
            c_col = b_col
            
            #add result to the output matrix if it's non-zero
            if c_val != 0:
                data.append(c_val)
                row.append(c_row)
                col.append(c_col)
    
    #COO matrix
    C = coo_matrix((data, (row, col)), shape=(A.shape[0], B.shape[1]))
    
    return C

In [4]:
A = rand(3, 4, density=0.4, format='coo')
B = rand(4, 2, density=0.4, format='coo')

C = mat_mul_coo(A, B)

print(C.toarray())

[[0.30644342 0.        ]
 [0.         0.21796054]
 [0.         0.04908339]]


implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [13]:
def mat_mul_csr(A, B):
    
    m, k1 = A.shape #rows and columns
    k2, n = B.shape
    
    if k1 != k2:
        raise ValueError("Matrices are not compatible for multiplication")

    dict_B = {}
    for i, row in enumerate(B):
        for j, val in zip(row.indices, row.data):
            if j not in dict_B:
                dict_B[j] = []
            dict_B[j].append((i, val))

    data = []
    indices = []
    indptr = [0]
    for i, row in enumerate(A):
        for j, val in zip(row.indices, row.data):
            if j not in dict_B:
                continue
            b_vals = [v for (r, v) in dict_B[j]]
            b_rows = [r for (r, v) in dict_B[j]]
            for b_val, b_row in zip(b_vals, b_rows):
                data.append(val * b_val)
                indices.append(b_row)
            indptr.append(len(indices))

    C = csr_matrix((data, indices, indptr), shape=(m, n))

    return C


In [14]:
A = csr_matrix([[1, 0, 2], [0, 3, 0], [4, 0, 5]])
B = csr_matrix([[6, 0], [0, 7], [8, 9]])

C = mat_mul_csr(A, B)

print(C.toarray())

[[ 6  0]
 [ 8 21]
 [51  0]]


implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

In [17]:
def solve_lin_sys(A, b):
    
    x = spsolve(A, b)
    
    return x

In [18]:
data = np.array([1, 2, 3, 4, 5])
indices = np.array([0, 1, 2, 0, 1])
indptr = np.array([0, 2, 4, 5])
A = csr_matrix((data, indices, indptr))

#right hand vector
b = np.array([6, 7, 8])

x = solve_lin_sys(A, b)

# Print the solution vector x
print(x)

[ 2.8  1.6 -1.4]
